<a href="https://colab.research.google.com/github/Dbroug01/training-data-analyst/blob/master/Json_to_BQ_schema_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
import json
import io
json_data = json.load(io.BytesIO(uploaded['most_recent_json.json']))

## How to access fields...

In [0]:
type(json_data['audits']['accesskeys'])

dict

## How to identify keys

In [0]:
json_data.keys()

dict_keys(['userAgent', 'environment', 'lighthouseVersion', 'fetchTime', 'requestedUrl', 'finalUrl', 'runWarnings', 'audits', 'configSettings', 'categories', 'categoryGroups', 'timing', 'i18n', 'stackPacks'])

## Steps for our crawler...


1.   load the json file using json.load
2.   Check which keys are available..
3. loop for every key and try:
3.1. is it single value or is it a nested object?
3.2. if value, what type is it?
3.3. create schema  object for BQ
3.4. if nested object, check if it is a repeated nest
3.5. dive into the nest and repeat





In [0]:
keys_dict = json_data.keys()
print(keys_dict)

dict_keys(['userAgent', 'environment', 'lighthouseVersion', 'fetchTime', 'requestedUrl', 'finalUrl', 'runWarnings', 'audits', 'configSettings', 'categories', 'categoryGroups', 'timing', 'i18n', 'stackPacks'])


In [0]:
json_data['i18n']

{'icuMessagePaths': {'lighthouse-core/audits/accessibility/accesskeys.js | description': ['audits.accesskeys.description'],
  'lighthouse-core/audits/accessibility/accesskeys.js | title': ['audits.accesskeys.title'],
  'lighthouse-core/audits/accessibility/aria-allowed-attr.js | description': ['audits[aria-allowed-attr].description'],
  'lighthouse-core/audits/accessibility/aria-allowed-attr.js | title': ['audits[aria-allowed-attr].title'],
  'lighthouse-core/audits/accessibility/aria-required-attr.js | description': ['audits[aria-required-attr].description'],
  'lighthouse-core/audits/accessibility/aria-required-attr.js | title': ['audits[aria-required-attr].title'],
  'lighthouse-core/audits/accessibility/aria-required-children.js | description': ['audits[aria-required-children].description'],
  'lighthouse-core/audits/accessibility/aria-required-children.js | title': ['audits[aria-required-children].title'],
  'lighthouse-core/audits/accessibility/aria-required-parent.js | descripti

In [0]:
for key in keys_dict:
  print(type(json_data[key]))

<class 'str'>
<class 'dict'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'list'>


## Loop function for only top layer

Objective is to build a set of functions that loops only through the highest layer in the json and construct the BQ schema. If it identifies layers of nesting it will only prepare the fields but not populate them just yet

In [0]:
# empty dictionary object that will turn into the BQ schema
bq_schema = {}

# reading in the json file
json_data = json.load(io.BytesIO(uploaded['most_recent_json.json']))
if type(json_data) == dict:
  bq_nest = nest_crawler(json_data)
  print(bq_nest)

main: userAgent
userAgent<class 'str'>
main: environment
main: networkUserAgent
networkUserAgent<class 'str'>
main: hostUserAgent
hostUserAgent<class 'str'>
main: benchmarkIndex
benchmarkIndex<class 'int'>
main: lighthouseVersion
lighthouseVersion<class 'str'>
main: fetchTime
fetchTime<class 'str'>
main: requestedUrl
requestedUrl<class 'str'>
main: finalUrl
finalUrl<class 'str'>
main: runWarnings
main: audits
main: is-on-https
main: id
id<class 'str'>
main: title
title<class 'str'>
main: description
description<class 'str'>
main: score
score<class 'int'>
main: scoreDisplayMode
scoreDisplayMode<class 'str'>
main: displayValue
displayValue<class 'str'>
main: details
main: type
type<class 'str'>
main: headings
main: items
main: redirects-http
main: id
id<class 'str'>
main: title
title<class 'str'>
main: description
description<class 'str'>
main: score
score<class 'int'>
main: scoreDisplayMode
scoreDisplayMode<class 'str'>
main: service-worker
main: id
id<class 'str'>
main: title
title<cla

In [0]:
def nest_crawler(json_snippet):
  bq_nest = []
  keys_dict = json_snippet.keys()
  for key in keys_dict:
    json_object = json_snippet[key]

    bq_snippet = construct_bq_snippet(key, type(json_object))
    bq_nest.append(bq_snippet)
  return bq_nest

In [0]:
def construct_bq_snippet(name, data_type):
  # converts python types into bq types
  # and constructs the individual bq_schema thingies
  
  # add for validation
  print(name + str(data_type))
  if data_type == int:
    data_type = "INTEGER"
    object_type = "NULLABLE"
    fields = False
  elif data_type == str:
    data_type = "STRING"
    object_type = "NULLABLE"
    fields = False
  elif data_type == float:
    data_type = "FLOAT"
    object_type = "NULLABLE"
    fields = False
  elif data_type == bool:
    data_type = "BOOLEAN"
    object_type = "NULLABLE"
    fields = False
  elif data_type == dict:
    data_type = "RECORD"
    object_type = "NULLABLE"
    fields = True
  elif data_type == list:
    data_type = "RECORD"
    object_type = "REPEATED"
    fields = True
  if fields is False:
    bq_snippet = {"name": name,
                 "type": data_type,
                 "mode": object_type}
  else: 
    bq_snippet = {"name": name,
                 "type": data_type,
                 "mode": object_type,
                 "fields": []}
    ## TODO: Figure out how to fill out the "fields"
  return bq_snippet

## Testing building multi-layer schema:

In [0]:
bq_schema = []    
bq_snippet = {"name": "something",
               "type": "INTEGER",
               "mode": "NULLABLE"}
bq_snippet2 = {"name": "something",
               "type": "INTEGER",
               "mode": "NULLABLE",
               "fields": []}
bq_snippet3 = {"name": "something",
               "type": "INTEGER",
               "mode": "NULLABLE"}
bq_snippet4 = {"name": "something",
               "type": "INTEGER",
               "mode": "NULLABLE"}
bq_schema.append(bq_snippet)
bq_schema.append(bq_snippet2)
bq_schema[1]['fields'].append(bq_snippet3)
bq_schema[1]['fields'].append(bq_snippet4)

In [0]:
bq_schema
# somehow the order of items changes... need to find a way of preserving the order. 
# This might be because it's a dict, and dicts are not ordered. 
# How come fields sits within its own separate dict?

[{'mode': 'NULLABLE', 'name': 'something', 'type': 'INTEGER'},
 {'fields': [{'mode': 'NULLABLE', 'name': 'something', 'type': 'INTEGER'},
   {'mode': 'NULLABLE', 'name': 'something', 'type': 'INTEGER'}],
  'mode': 'NULLABLE',
  'name': 'something',
  'type': 'INTEGER'}]

In [0]:
# for constructing nested fields, need to think if this is the best way of doing this... maybe before adding bq_snippet2 to the main, we could keep adding stuff to bq_snippet2['fields']

In [0]:
bq_snippet2['fields']

[{'mode': 'NULLABLE', 'name': 'something', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'something', 'type': 'INTEGER'}]

## Version two of loop function

This time try to also go into the nests and construct the schema for those

In [0]:
def nest_crawler(json_snippet):
  bq_nest = []
  try:
    keys_dict = json_snippet.keys()

    for key in keys_dict:
      # add for validation
      #print("main: " + str(key))
      json_object = json_snippet[key]

      if type(json_object) == dict:
        # if dict it is probably a nested object
        bq_nest_in_nest = nest_crawler(json_object)
        bq_nest.append(bq_nest_in_nest)
      elif type(json_object) == list:
        # if it is a list, it is probably a repeated nest... we can basically do stuff...
        bq_nest_in_nest = nest_crawler(json_object)
        bq_nest.append(bq_nest_in_nest)
      else:
        # if it is none of the previous ones then it must be a value field
        bq_snippet = construct_bq_snippet(key, type(json_object))
        bq_nest.append(bq_snippet)
    return bq_nest
  except AttributeError:
    bq_nest = []
    return bq_nest
  
  # the try, except has been added to deal with cases where a dict is empty and therefore has no keys.
  # in that case, just return an empty array...
  # however need to see if this is a feasible solution.... 
  # big TODO: how would it look like if it wasn't empty?

## Version three

Build add something that deals with NoneTypes

In [0]:
def nest_crawler(json_snippet):
  bq_nest = []
  try:
    keys_dict = json_snippet.keys()

    for key in keys_dict:
      # add for validation
      print("main: " + str(key))
      json_object = json_snippet[key]

      if type(json_object) == dict:
        # if dict it is probably a nested object
        bq_nest_in_nest = nest_crawler(json_object)
        bq_nest.append(bq_nest_in_nest)
      elif type(json_object) == list:
        # if it is a list, it is probably a repeated nest... we can basically do stuff...
        bq_nest_in_nest = nest_crawler(json_object)
        bq_nest.append(bq_nest_in_nest)
      elif json_object is None:
        # so far NoneTypes have only been identified for score fields
        # for now, simply declare them as FLOATs
        bq_snippet = construct_bq_snippet(key, type(1.0))
        bq_nest.append(bq_snippet)
      else:
        # if it is none of the previous ones then it must be a value field
        bq_snippet = construct_bq_snippet(key, type(json_object))
        bq_nest.append(bq_snippet)
    return bq_nest
  except AttributeError:
    bq_nest = []
    return bq_nest
  
  # it is now able to run through to the end but it is not dealing correctly with the lists and dicts. 
  #The part where it is adding these to the fields just doesn't work.. the whole thing just turns out as []
  # try to build a mini json with some straightforward nesting and then check the code line by line with a validator...

## Example of BigQuery schema with nests

In [0]:
[
    {
        "name": "something",
        "type": "STRING",
        "mode": "NULLABLE"
    },
    {
        "name": "hello",
        "type": "RECORD",
        "mode": "REPEATED",
        "fields": [
            {
                "name": "something_else",
                "type": "RECORD",
                "mode": "NULLABLE",
                "fields": [
                    {
                        "name": "something_again",
                        "type": "STRING",
                        "mode": "NULLABLE"
                    },
                    {
                        "name": "another_item",
                        "type": "INTEGER",
                        "mode": "NULLABLE"
                    }
                ]
            }
        ]
    }
]

[{'mode': 'NULLABLE', 'name': 'something', 'type': 'STRING'},
 {'fields': [{'fields': [{'mode': 'NULLABLE',
      'name': 'something_again',
      'type': 'STRING'},
     {'mode': 'NULLABLE', 'name': 'another_item', 'type': 'INTEGER'}],
    'mode': 'NULLABLE',
    'name': 'something_else',
    'type': 'RECORD'}],
  'mode': 'REPEATED',
  'name': 'hello',
  'type': 'RECORD'}]